In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras import layers, models, optimizers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os
import tensorflow as tf
from keras.callbacks import TensorBoard
import time
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
base_train_dir = 'xray1200/train'
base_validation_dir = 'xray1200/test/'
base_test_dir = 'xray1200/val'

train_normal_dir = 'xray1200/train/NORMAL'
train_pneumonia_dir = 'xray1200/train/PNEUMONIA'

validation_normal_dir = 'xray1200/test/NORMAL'
validation_pneumonia_dir = 'xray1200/test/PNEUMONIA'

test_normal_dir = 'xray1200/val/NORMAL'
test_pneumonia_dir = 'xray1200/val/PNEUMONIA'

print('total traing NORMAL images:', len(os.listdir(train_normal_dir)))
print('total traing PNEUMONIA images:', len(os.listdir(train_pneumonia_dir)))
print()
print('total validation NORMAL images:', len(os.listdir(validation_normal_dir)))
print('total validation PNEUMONIA images:', len(os.listdir(validation_pneumonia_dir)))
print()
print('total test NORMAL images:', len(os.listdir(test_normal_dir)))
print('total test PNEUMONIA images:', len(os.listdir(test_pneumonia_dir)))
print()

# преобразование значений пикселей к 1./255
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  zoom_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        base_train_dir, 
        target_size=(150, 150), #размер изображения
        batch_size=20, #размер пакета
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        base_validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

print()
print(train_generator.class_indices)

total traing NORMAL images: 1200
total traing PNEUMONIA images: 1200

total validation NORMAL images: 300
total validation PNEUMONIA images: 300

total test NORMAL images: 83
total test PNEUMONIA images: 119

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.

{'NORMAL': 0, 'PNEUMONIA': 1}


In [6]:
dense_layers = [0, 1, 2]
layer_sizes = [64, 128, 256]
conv_layers = [1, 2, 3, 4]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, 
                                                         layer_size, 
                                                         dense_layer, 
                                                         int(time.time()))

            model = models.Sequential()
            
            model.add(layers.Conv2D(32, (3, 3), activation='relu',
                                    input_shape=(150, 150, 3)))
            model.add(layers.MaxPooling2D((2, 2)))
            
            for layer in range(conv_layer-1):
                model.add(layers.Conv2D(layer_size, (3, 3), activation='relu'))
                model.add(layers.MaxPooling2D((2, 2)))
               
            model.add(layers.Flatten())
            model.add(layers.Dropout(0.5))
            
            for layer in range(dense_layer):
                model.add(layers.Dense(layer_size, activation='relu'))
                
            model.add(layers.Dense(1, activation='sigmoid'))
            
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            
            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['acc'])
            
            model.fit_generator(
                                train_generator,
                                steps_per_epoch=120,
                                epochs=15,
                                validation_data=validation_generator,
                                validation_steps=30,
                                callbacks=[tensorboard])
                     

#model.summary()

Epoch 1/15
120/120 [==============================] - 47s 391ms/step - loss: 0.6559 - acc: 0.7304 - val_loss: 0.4165 - val_acc: 0.8083
Epoch 2/15
120/120 [==============================] - 42s 346ms/step - loss: 0.3649 - acc: 0.8446 - val_loss: 0.3705 - val_acc: 0.8467
Epoch 3/15
120/120 [==============================] - 43s 356ms/step - loss: 0.3084 - acc: 0.8762 - val_loss: 0.3417 - val_acc: 0.8617
Epoch 4/15
120/120 [==============================] - 42s 351ms/step - loss: 0.2813 - acc: 0.8854 - val_loss: 0.3557 - val_acc: 0.8617
Epoch 5/15
120/120 [==============================] - 44s 367ms/step - loss: 0.2503 - acc: 0.8979 - val_loss: 0.3507 - val_acc: 0.8600
Epoch 6/15
120/120 [==============================] - 44s 364ms/step - loss: 0.2396 - acc: 0.9054 - val_loss: 0.3620 - val_acc: 0.8633
Epoch 7/15
120/120 [==============================] - 43s 359ms/step - loss: 0.2217 - acc: 0.9150 - val_loss: 0.5635 - val_acc: 0.7817
Epoch 8/15
120/120 [==============================] - 4

120/120 [==============================] - 44s 370ms/step - loss: 0.5499 - acc: 0.7408 - val_loss: 0.4040 - val_acc: 0.8133
Epoch 2/15
120/120 [==============================] - 41s 341ms/step - loss: 0.3138 - acc: 0.8725 - val_loss: 0.3371 - val_acc: 0.8617
Epoch 3/15
120/120 [==============================] - 43s 357ms/step - loss: 0.2980 - acc: 0.8758 - val_loss: 0.4037 - val_acc: 0.8200
Epoch 4/15
120/120 [==============================] - 43s 362ms/step - loss: 0.2363 - acc: 0.9112 - val_loss: 0.4705 - val_acc: 0.7967
Epoch 5/15
120/120 [==============================] - 43s 361ms/step - loss: 0.2602 - acc: 0.8929 - val_loss: 0.4293 - val_acc: 0.8050
Epoch 6/15
120/120 [==============================] - 43s 362ms/step - loss: 0.2507 - acc: 0.9033 - val_loss: 0.4121 - val_acc: 0.8350
Epoch 7/15
120/120 [==============================] - 44s 363ms/step - loss: 0.2240 - acc: 0.9117 - val_loss: 0.5304 - val_acc: 0.7833
Epoch 8/15
120/120 [==============================] - 43s 359ms/st

KeyboardInterrupt: 

In [40]:
model.save('xray_2.h5')

In [41]:
def prepare(filepath):
    IMG_SIZE = 150  
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # черно-белое  изображдение
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # размер изображения
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

model = tf.keras.models.load_model("xray_2.h5")

count_normal = 0
count_pneumo = 0

for img in os.listdir(test_normal_dir):
    prep_img = image.load_img(test_normal_dir+'/'+img, target_size=(150, 150))
    x = image.img_to_array(prep_img)
    x = x.reshape((1,) + x.shape)
    prediction = model.predict(x)
    if prediction == 0:
        count_normal +=1

print(str(count_normal) + ' from ' + str(len(os.listdir(test_normal_dir))))

    
for img in os.listdir(test_pneumonia_dir):
    prep_img = image.load_img(test_pneumonia_dir+'/'+img, target_size=(150, 150))
    x = image.img_to_array(prep_img)
    x = x.reshape((1,) + x.shape)
    prediction = model.predict(x)
    if prediction == 1:
        count_pneumo +=1
    
print(str(count_pneumo) + ' from ' + str(len(os.listdir(test_pneumonia_dir))))